In [1]:
#!pip install -q -U git+https://github.com/fabienfrfr/tptt@main
#!python -V
#!pip list

- Llama / Llama 2/3	`self_attn ou attention`	q_proj, k_proj, v_proj, o_proj
- GPT-2/3/Neo/J	`attn ou attention`	c_attn, c_proj (GPT-2), q_proj, etc.
- Mistral	`self_attn`	q_proj, k_proj, v_proj, o_proj
- OpenELM	`attn`	qkv_proj, out_proj

In [ ]:
"""
hidden_states.shape: torch.Size([2, 256, 2048])
attention_mask.shape: torch.Size([2, 1, 256, 256])
v.shape: torch.Size([2, 256, 256])
"""

In [ ]:
import sys
sys.path.append('../src')

import tptt
from datasets import load_dataset

# 1. Config et modèle
# config = tptt.TpttConfig(base_model_name="gpt2") # Target modules {'v_proj', 'q_proj'} not found in the base model.
# config = tptt.TpttConfig(base_model_name="TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T") # Minimum 5GB RAM (if not quantized, else 500MB)
config = tptt.TpttConfig(base_model_name="apple/OpenELM-270M", base_tokenizer_name="hf-internal-testing/llama-tokenizer")
model = tptt.TpttModel(config)

# 2. (Optionnel) Injection LoRA
model.add_lora()

# 3. Préparation du dataset
dataset = load_dataset("yahma/alpaca-cleaned")["train"].select(range(100))
# (instruction_format est importé via tptt)
dataset = dataset.map(tptt.instruction_format)

# 4. Entraînement
trainer = tptt.TpttTrainer(model, dataset)
trainer.train()

# 5. Génération
pipe = tptt.TpttPipeline(model)
print(pipe("Once upon a time,"))

# 6. Sauvegarde
model.save_pretrained("./my_tptt_model")

/home/fabien/.pyenv/versions/3.11.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
